In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
!pip install gradio
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.5 MB/

In [3]:
import gradio as gr
import numpy as np
import torch

In [4]:
def load_model(model_config,processor,checkpoint_path,device):
  # model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
  # processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
  # Create an instance of the model architecture with the loaded configuration
  sidewalk_model = SamModel(config=model_config)
  checkpoint = torch.load(checkpoint_path)
  sidewalk_model.load_state_dict(checkpoint['model_state'])
  # device = "cuda" if torch.cuda.is_available() else "cpu"
  sidewalk_model.to(device)
  return sidewalk_model

In [5]:
from transformers import SamModel, SamConfig, SamProcessor

checkpoint_path="/content/drive/MyDrive/SAM/checkpoint.pth_1"
device = "cuda" if torch.cuda.is_available() else "cpu"
model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
SAM_processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
sidewalk_model=load_model(model_config,SAM_processor,checkpoint_path,device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [6]:
from transformers import SamModel, SamConfig, SamProcessor

def SidewalkPrompter(sidewalk_model,processor,image_np,prompt_points,prompt_points_labels,prompt_boxes,threshold):
  """
  - SAM_model:
  - image_np:
  - prompt_points: points visual prompt in one mask in format of [[x1,y1],[x2,y2],[x3,y3],..,[xn,yn]]
  - prompt_points_labels : labels for prompt_points --> 1: forground 0:background
  - prompt_boxes: area of interest in bbox format : [x_left,y_top,x_right,y_buttom]
  - threshold: confidence of the masks
  """
  input_points=[prompt_points]
  input_labels=[prompt_points_labels]
  # input_boxes=[[prompt_boxes]]
  if prompt_boxes is not None:
    input_boxes=[[prompt_boxes]]
    inputs = processor(image_np, input_boxes=input_boxes,input_points=input_points,input_labels=input_labels,return_tensors="pt")
  else:
    inputs = processor(image_np,input_points=input_points,input_labels=input_labels,return_tensors="pt")

  device = "cuda" if torch.cuda.is_available() else "cpu"
  inputs = {k: v.to(device) for k, v in inputs.items()}
  sidewalk_model.eval()
  with torch.no_grad():
    outputs = sidewalk_model(**inputs, multimask_output=False)

    # apply sigmoid
  prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
  # convert soft mask to hard mask
  seg_prob = prob.cpu().numpy().squeeze()
  mask= (seg_prob > threshold).astype(np.uint8)

  return mask

In [7]:
from PIL import Image
import cv2

def preprocess(image):
  """
  Make sure the image is a numpy array of size (256,256)
  """
  # image= np.array(Image.open(image_path))
  image= np.array(image)
  image_resize=cv2.resize(image, (256, 256))
  return image_resize

In [8]:
import matplotlib.pyplot as plt
def overlay_mask(image,mask,show=True):
  overlay = np.zeros_like(image)
  # color = np.array([0,200,255], dtype=np.uint8)
  color = np.array([255,215,0], dtype=np.uint8)
  overlay[mask == 1] =color
  alpha = 0.5
  combined = cv2.addWeighted(image, 1, overlay,alpha , 0)
  if show:
    plt.figure(figsize=(5,5))
    plt.imshow(combined)
    plt.axis('off')
    plt.show()
  return combined

In [10]:
import json
def segment_image(image, bbox_input,points_input):
  """
  image: PIL objects
  bbox: bbounding box selescted by the user [x_min, y_min, x_max, y_max]
  points: points selescted by the user [(x1, y1), (x2, y2), ...]

  """
  bbox= json.loads(bbox_input)
  points = json.loads(points_input)
  image=preprocess(image)
  num_points = len(points)
  lables=[]
  for i in range(num_points):
    lables.append(1)
  # print(lables)
  mask=SidewalkPrompter(sidewalk_model,SAM_processor,image,points,lables,bbox,0.5)

  seg_prompt=overlay_mask(image,mask,show=False)

  return seg_prompt

In [11]:
iface = gr.Interface(
    fn=segment_image,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Textbox(label="Enter bounding box prompt", placeholder="[x_min, y_min, x_max, y_max]", lines=1),
        gr.Textbox(label="Enter point prompt", placeholder="[[x1, y1], [x2, y2], ...]", lines=3)
    ],
    outputs=[
        gr.Image(label="Segmented Image"),
    ],
    title="Sidewalk Segmentor",
    description="- Upload an sidewalk image\n - Enter corresponding bounding box, points coordinates as prompts\n - Wait for the sidewalk segmentation result"
)
iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3a15fe3eec7b3bfbbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3a15fe3eec7b3bfbbc.gradio.live
